In [8]:
#pip install torchvision

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 14.2 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 105.5 MB/s eta 0:00:01
     |████████████████████████████████| 218.8 MB 114.5 MB/s eta 0:00:01  |███████▎                        | 49.4 MB 8.0 MB/s eta 0:00:22
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussio

In [9]:
import numpy as np
import torch

Lets play with pytorch a little bit!

In [10]:
# Create tensors.
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [11]:
print(x)

tensor(3.)


In [12]:
print(w)

tensor(4., requires_grad=True)


In [13]:
print(b)

tensor(5., requires_grad=True)


In [14]:
y = w*x+b
print(y)

tensor(17., grad_fn=<AddBackward0>)


What makes PyTorch special, is that we can automatically compute the derivative of y w.r.t. the tensors that have requires_grad set to True i.e. w and b.

In [15]:
y.backward() #compute gradients

In [16]:
print('dy/dw:',w.grad) #display gradients
print('dy/db:',b.grad)#display gradients

dy/dw: tensor(3.)
dy/db: tensor(1.)


# Now is a real problem

We'll create a model that predicts crop yeilds for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region. 

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yeild_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2


Our objective: Find a suitable set of weights and biases using the training data, to make accurate predictions

### Data

In [19]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [20]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [21]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Linear Regression Model (from scratch)

The weights and biases can also be represented as matrices, initialized with random values. The first row of w and the first element of b are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [23]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.1623, -1.9893, -0.5136],
        [ 0.2625,  0.9617, -1.7172]], requires_grad=True)
tensor([1.5063, 0.5398], requires_grad=True)


The model is simply a function that performs a matrix multiplication of the input x and the weights w (transposed) and adds the bias b (replicated for each observation).



In [24]:
# Define the model
def model(x):
    return x @ w.t() + b

In [25]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-165.7104,   10.2922],
        [-221.1931,   -0.8497],
        [-308.9682,   52.6415],
        [-119.5939,    5.1264],
        [-236.6178,   -9.2336]], grad_fn=<AddBackward0>)


In [26]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Because we've started with random weights and biases, the model does not a very good job of predicting the target varaibles.

### Loss Function
We can compare the predictions with the actual targets, using the following method:

Calculate the difference between the two matrices (preds and targets).
Square all elements of the difference matrix to remove negative values.
Calculate the average of the elements in the resulting matrix.
The result is a single number, known as the mean squared error (MSE).

In [27]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [28]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(49687.7734, grad_fn=<DivBackward0>)


The resulting number is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

### Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True.

In [29]:
# Compute gradients
loss.backward()

In [30]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.1623, -1.9893, -0.5136],
        [ 0.2625,  0.9617, -1.7172]], requires_grad=True)
tensor([[-23758.7754, -27497.4375, -16541.6562],
        [ -6543.4824,  -7482.4448,  -4780.4570]])


In [31]:
# Gradients for bias
print(b)
print(b.grad)

tensor([1.5063, 0.5398], requires_grad=True)
tensor([-286.6167,  -80.4046])


A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is postive,
increasing the element's value slightly will increase the loss.
decreasing the element's value slightly will decrease the loss.

If a gradient element is negative,
increasing the element's value slightly will decrease the loss.
decreasing the element's value slightly will increase the loss.

The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [32]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1- Generate predictions

2- Calculate the loss

3- Compute gradients w.r.t the weights and biases

4- Adjust the weights by subtracting a small quantity proportional to the gradient

5- Reset the gradients to zero

In [33]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-165.7104,   10.2922],
        [-221.1931,   -0.8497],
        [-308.9682,   52.6415],
        [-119.5939,    5.1264],
        [-236.6178,   -9.2336]], grad_fn=<AddBackward0>)


In [34]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(49687.7734, grad_fn=<DivBackward0>)


In [35]:
# Compute gradients
loss.backward()

In [36]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [37]:
print(w)


tensor([[ 0.0753, -1.7143, -0.3482],
        [ 0.3279,  1.0365, -1.6694]], requires_grad=True)


In [38]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(34063.0469, grad_fn=<DivBackward0>)


### Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [39]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [40]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(679.1196, grad_fn=<DivBackward0>)


In [41]:
# Print predictions
preds

tensor([[ 66.3564,  74.4609],
        [ 88.1411,  86.3452],
        [ 90.4253, 158.8649],
        [ 72.9992,  58.8916],
        [ 82.2460,  81.5582]], grad_fn=<AddBackward0>)

In [42]:
# Print targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### Linear Regression Model using PyTorch built-ins

Let's re-implement the same model using some built-in functions and classes from PyTorch.

In [43]:
# Imports
import torch.nn as nn

In [47]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [50]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [51]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [52]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [53]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[ 69.,  96.,  70.],
         [ 91.,  88.,  64.],
         [102.,  43.,  37.],
         [ 87., 134.,  58.],
         [ 91.,  88.,  64.]]),
 tensor([[103., 119.],
         [ 81., 101.],
         [ 22.,  37.],
         [119., 133.],
         [ 81., 101.]])]

### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using nn.Linear

In [54]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2081, -0.3960, -0.4368],
        [-0.3419,  0.0262,  0.5476]], requires_grad=True)
Parameter containing:
tensor([-0.2675,  0.3396], requires_grad=True)


### Optimizer

Instead of manually manipulating the weights & biases using gradients, we can use the optimizer optim.SGD

In [55]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Loss Function

Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [56]:
# Import nn.functional
import torch.nn.functional as F

In [57]:
# Define loss function
loss_fn = F.mse_loss

In [58]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(12401.4365, grad_fn=<MseLossBackward>)


### Train the model

We are ready to train the model now. We can define a utility function fit which trains the model for a given number of epochs.



In [59]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(inputs), targets))

In [60]:
# Train the model for 100 epochs
fit(100, model, loss_fn, opt)

Training loss:  tensor(28.0611, grad_fn=<MseLossBackward>)


In [61]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 59.0768,  70.9656],
        [ 79.7396, 100.8391],
        [120.9883, 131.4860],
        [ 32.2651,  40.4978],
        [ 91.0668, 117.4694],
        [ 59.0768,  70.9656],
        [ 79.7396, 100.8391],
        [120.9883, 131.4860],
        [ 32.2651,  40.4978],
        [ 91.0668, 117.4694],
        [ 59.0768,  70.9656],
        [ 79.7396, 100.8391],
        [120.9883, 131.4860],
        [ 32.2651,  40.4978],
        [ 91.0668, 117.4694]], grad_fn=<AddmmBackward>)

In [62]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Bonus: Feedfoward Neural Network

Conceptually, you think of feedforward neural networks as two or more linear regression models stacked on top of one another with a non-linear activation function applied between them.


To use a feedforward neural network instead of linear regression, we can extend the nn.Module class from PyTorch.

In [63]:
class SimpleNet(nn.Module):
    # Initialize the layers
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 3)
        self.act1 = nn.ReLU() # Activation function
        self.linear2 = nn.Linear(3, 2)
    
    # Perform the computation
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        return x

Now we can define the model, optimizer and loss function exactly as before.



In [64]:
model = SimpleNet()
opt = torch.optim.SGD(model.parameters(), 1e-5)
loss_fn = F.mse_loss


In [65]:
fit(100, model, loss_fn, opt)


Training loss:  tensor(18.3240, grad_fn=<MseLossBackward>)
